<a href="https://colab.research.google.com/github/ErokhinVD111/Parsing/blob/main/ParsingHSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [32]:
# Задание №1
url = 'https://hmat.hse.ru/persons'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [34]:
persons = soup.find_all('a', class_='fa-person__name')
employee_links = []

In [35]:
for person in persons:
    parsed_person = (
        str(person).replace('\t', '')
        .replace('\n', ' ')
        .replace('<a class="fa-person__name" href="', '')
        .replace('">', '')
        .replace('</a>', '')
        .split()
    )
    name = f'{parsed_person[1]} {parsed_person[2]} {parsed_person[3]}'
    link = f'{parsed_person[0]}'
    employees[name] = f"https:{link}"

In [36]:
links = list(employees.values())

In [37]:
employees

{'Боброва Ирина Александровна': 'https://www.hse.ru/staff/iabobrova',
 'Буров Александр Анатольевич': 'https://www.hse.ru/org/persons/134016834',
 'Гончаренко Василий Михайлович': 'https://www.hse.ru/org/persons/190260375',
 'Дагаев Дмитрий Александрович': 'https://www.hse.ru/staff/ddagaev',
 'Дружининская Ирина Михайловна': 'https://www.hse.ru/org/persons/135878',
 'Елаева Мария Сергеевна': 'https://www.hse.ru/org/persons/224066299',
 'Кийков Сергей Ортабаевич': 'https://www.hse.ru/org/persons/412677111',
 'Колотвина Оксана Альбертовна': 'https://www.hse.ru/org/persons/63501654',
 'Кочергин Вадим Васильевич': 'https://www.hse.ru/org/persons/223680621',
 'Латышев Александр Михайлович': 'https://www.hse.ru/org/persons/207912918',
 'Логвенков Сергей Алексеевич': 'https://www.hse.ru/org/persons/135936',
 'Макаров Алексей Алексеевич': 'https://www.hse.ru/org/persons/135897',
 'Малютина Полина Александровна': 'https://www.hse.ru/staff/pamalyutina',
 'Михайлович Анна Витальевна': 'https://ww

In [38]:
#Задание №2
url = links[11]
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [39]:
employee_data = []
page = soup.find_all('div', class_='main')
for info in page:
    employee_data.append(info.find('h1').text)
    employee_data.append(int(info.find('li', class_='i').text.split()[6]))

In [40]:
page = soup.find_all('dl', class_='main-list large main-list-language-knowledge-level')
for info in page:
    languages = ( 
        str(info).replace('<dl class="main-list large main-list-language-'
        'knowledge-level"><dt class="b">Владение языками</dt>', '') 
        .replace('<dd>', '')
        .replace('</dd>', ' ')
        .replace('</dl>', '')
        .split()
    )
    employee_data.append(' '.join(languages))

In [41]:
employee_data

['Сысоева Любовь Николаевна', 2016, 'английский немецкий']

In [42]:
# Задание №3
def hse_info(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'lxml')

    employee_data = []
    page = soup.find_all('div', class_='main')
    for info in page:
        employee_data.append(info.find('h1').text)
        employee_data.append(int(info.find('li', class_='i').text.split()[6]))

    page = soup.find_all('dl', class_='main-list large main-list-language-knowledge-level')
    for info in page:
        languages = (
            str(info).replace('<dl class="main-list large main-list-language-'
            'knowledge-level"><dt class="b">Владение языками</dt>', '')
            .replace('<dd>', '')
            .replace('</dd>', ' ')
            .replace('</dl>', '')
            .split()
        )
        employee_data.append(' '.join(languages))

    if not page:
        employee_data.append('нет данных')

    return employee_data

In [43]:
hse_info(links[7])

['Сальникова Дарья Вячеславовна', 2011, 'английский']

In [44]:
# Задание №4
employees = []
for link in links:
    employees.append(hse_info(link))

In [45]:
employees

[['Гончаренко Василий Михайлович', 2016, 'английский'],
 ['Дагаев Дмитрий Александрович', 2010, 'английский французский'],
 ['Елаева Мария Сергеевна', 2018, 'английский'],
 ['Кийков Сергей Ортабаевич', 2020, 'английский французский'],
 ['Логвенков Сергей Алексеевич', 2000, 'английский'],
 ['Михайлович Анна Витальевна', 2011, 'английский французский немецкий'],
 ['Никонов Василий Иванович', 2018, 'английский'],
 ['Сальникова Дарья Вячеславовна', 2011, 'английский'],
 ['Сахарова Нина Евгеньевна', 2010, 'английский'],
 ['Симонова Галина Ивановна', 2007, 'английский'],
 ['Стукал Денис Константинович', 2008, 'английский испанский немецкий'],
 ['Сысоева Любовь Николаевна', 2016, 'английский немецкий'],
 ['Филимонов Дмитрий Андреевич',
  2013,
  'английский французский немецкий испанский'],
 ['Шнурков Петр Викторович', 2012, 'английский'],
 ['Щуров Илья Валерьевич', 2010, 'английский'],
 ['Латышев Александр Михайлович', 2020, 'нет данных'],
 ['Никонова Екатерина Александровна', 2019, 'английс

In [48]:
df = pd.DataFrame(data=employees, columns=['ФИО', 'Год начала работы', 'Владение языками'])
with pd.ExcelWriter("HSE.xlsx") as writer:
    df.to_excel(writer, index=False)